# Reflexive Verbs

Gotta love the lack of documentation about obscure languages. Lets scrape the conjugation rules from the internet.

## First scrape a bunch of reflexive verb tables

In [2]:
from bs4 import BeautifulSoup
import requests
from pprint import pprint

def get_soup(suffix, base_url='https://en.wiktionary.org'):
    return BeautifulSoup(requests.get(base_url + suffix).text, 'lxml')

words = {}

base_soup = get_soup('/wiki/Category:Latvian_verbs')
for word_soup in base_soup.find_all('div', class_='mw-content-ltr')[0].find_all('a', href=True):
    if word_soup.has_attr('title'):
        try:
            conjug_soup = get_soup(word_soup['href']).find_all('table', class_='inflection-table')
            data = [
                [
                    {"string":ele.text.strip(),
                     "type": "th",
                     "rowspan": int(ele['rowspan']) if ele.has_attr('rowspan') else 1,
                     "colspan": int(ele['colspan']) if ele.has_attr('colspan') else 1}
                    for ele in row.find_all('th')
                    #if ele.text.strip()
                ] + [
                    {"string":ele.text.strip(),
                     "type": "td",
                     "rowspan": int(ele['rowspan']) if ele.has_attr('rowspan') else 1,
                     "colspan": int(ele['colspan']) if ele.has_attr('colspan') else 1}
                    for ele in row.find_all('td')
                    #if ele.text.strip()
                ]
                for row in conjug_soup[0].find_all('tr')
            ]
            words[word_soup.text] = data
        except:
            pass
#pprint(words)

In [3]:
print(len(words))

203


## Try to extract out how the stem is used

In [4]:
import matplotlib.pyplot as plt
import numpy as np

def find_stem(word, word_block):
    conjugated_forms = ''.join([
        col['string']
        for row in word_block
        for col in row
        if col['type'] == 'td'
    ])
    
    stem_length_weight = 2.5
    occurances = np.array([
        conjugated_forms.count(word[:i]) * (i+1)**stem_length_weight
        for i in range(len(word))
    ])
    
    return word[:np.argmax(occurances)], word[np.argmax(occurances):]

def replace_stem(word_block, stem):
    return [
        [ 
            { key: 
                value.replace(stem, '_'*4) 
                if key == 'string' else value
                for key, value in col.items() }
            if col['type'] == 'td' else col
            for col in row
        ]
        for row in word_block
    ]

rule_blocks = {}
for word in words:
    stem, ending = find_stem(word, words[word])
    rule_blocks[f'[{stem}]{ending}'] = replace_stem(words[word], stem)

#pprint(rule_blocks)

Collapse down the rules

In [5]:
summarized_rules = [
    # ([words, ...], rule_block)
]
for word, rule in rule_blocks.items():
    try:
        index = list(zip(*summarized_rules))[1].index(rule)
        summarized_rules[index][0].append(word)
    except:
        summarized_rules.append(([word], rule))

## Now we can display our stem table

In [8]:
from IPython.core.display import display, HTML

def make_html(table_json):
    soup = BeautifulSoup('<table></table>', 'lxml')
    table = soup.table
    for row in table_json:
        tr = soup.new_tag('tr')
        for col in row:
            td = soup.new_tag(col['type'], **col)
            td.string = col['string']
            tr.append(td)
        table.append(tr)
    return str(soup)

for words, rule in sorted(summarized_rules, key=lambda x: -len(x[0])):
    if len(words) > 5:
        print(', '.join(words))
        display(HTML(make_html(rule)))

[vēro]t, [ievēro]t, [mērcē]t, [baro]t, [abstrahē]t, [akcentē]t, [aktivizē]t, [aktualizē]t, [apbrīno]t, [apvieno]t, [atvaino]t, [audzē]t, [auklē]t, [biedē]t, [bloķē]t, [brokasto]t, [burbuļo]t, [bučo]t, [centralizē]t, [centrē]t, [dabū]t, [deaktivizē]t, [dejo]t, [demokratizē]t, [deso]t, [diskutē]t, [domā]t, [dzemdē]t, [dzīvo]t, [ejakulē]t, [elpo]t, [fotografē]t, [garšo]t, [gatavo]t, [gādā]t, [indē]t, [iniciē]t, [inspirē]t, [intonē]t, [iztulko]t, [jautā]t, [kaplē]t, [kapā]t, [klepo]t, [konstatē]t, [kārto]t, [lido]t, [maksā]t, [mazgā]t, [meklē]t, [mezglo]t, [mežģī]t, [modernizē]t


[main]īt, [apmain]īt, [atstāst]īt, [baid]īt, [dal]īt, [dar]īt, [gaid]īt, [groz]īt, [izdar]īt, [izskat]īt, [klaus]īt, [laiz]īt, [las]īt, [lāp]īt, [man]īt, [māc]īt, [mān]īt


[zvēr]ēt, [prec]ēt, [cer]ēt, [mīl]ēt, [aizstāv]ēt, [atbild]ēt, [drīkst]ēt, [dzird]ēt, [grib]ēt, [kust]ēt, [līdz]ēt, [min]ēt


[aicin]āt, [audzin]āt, [bagātin]āt, [biedin]āt, [biezin]āt, [cepin]āt, [dancin]āt, [dievin]āt, [drošin]āt, [jautrin]āt, [kurin]āt, [mazin]āt


[aizar]t, [apcep]t, [atar]t, [atcep]t, [cep]t, [iear]t, [iecep]t, [izar]t, [izcep]t


[melo]t, [ceļo]t, [danco]t, [eksistē]t, [joko]t, [karo]t, [kāsē]t


[aizlie]gt, [bei]gt, [ieslē]gt, [izslē]gt, [jē]gt, [lie]gt, [lū]gt
